<a href="https://colab.research.google.com/github/1s5ac/STA365_HW/blob/main/STA365_HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pymc as pm
import numpy as np
import arviz as az

# Load the dataset
url = 'https://raw.githubusercontent.com/1s5ac/STA365_HW/main/psd_coffee.csv'
data = pd.read_csv(url)


# Part 1
n,p=100,10; X,y=np.zeros((n,p)),np.ones((n,1))
with pm.Model() as logistic_model:
    # Priors for the regression coefficients
    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))
    # Linear combination of features and weights
    linear_comb = pm.math.dot(X, betas)
    # Using a logit link function
    p = pm.math.sigmoid(linear_comb)
    # Bernoulli likelihood
    y_obs = pm.Bernoulli('y_obs', p=p, observed=y)
with logistic_model:
    idata = pm.sample()

# Part 2
n_samples = 100
n_features = 5

X = np.random.randn(n_samples, n_features)
y = np.random.randn(n_samples)
beta = np.random.randn(n_features)

# Hyperparameter lambda for the penalty terms
lambda_ = 1.0

# Ridge Regression loss function
def ridge_loss(X, y, beta, lambda_):
    prediction_error = np.sum((y - X.dot(beta)) ** 2)
    regularization = lambda_ * np.sum(beta ** 2)
    return 0.5 * prediction_error + regularization

# Lasso Regression loss function
def lasso_loss(X, y, beta, lambda_):
    prediction_error = np.sum((y - X.dot(beta)) ** 2)
    regularization = lambda_ * np.sum(np.abs(beta))
    return 0.5 * prediction_error + regularization

# Calculate the loss for both Ridge and Lasso
ridge_reg_loss = ridge_loss(X, y, beta, lambda_)
lasso_reg_loss = lasso_loss(X, y, beta, lambda_)

# Print the losses
print("Ridge Regression Loss:", ridge_reg_loss)
print("Lasso Regression Loss:", lasso_reg_loss)
print("Bayesians do not optimize posterior distributions, they sample from them; but, the posterior distributions are nonetheless 'regularizations' of the likelihood through the prior.")


# Part 3
np.random.seed(42)
n_samples = 100
X = np.random.normal(size=(n_samples, 1))
true_beta = 2.5
mu = X * true_beta
w = 1
nu = 4

# Simulating y with a normal distribution (for simplicity)
y = np.random.normal(mu.flatten(), scale=w)

# Robust regression model with explicit lambda modeling
with pm.Model() as robust_model_with_lambda:
    # Priors
    beta = pm.Normal('beta', mu=0, sigma=10)
    nu = pm.Exponential('nu', 1/29) + 1
    lambda_i = pm.Gamma('lambda', alpha=nu/2, beta=nu/2, shape=n_samples)
    sigma_i = pm.Deterministic('sigma_i', 1 / (lambda_i))

    # Likelihood
    likelihood = pm.Normal('y', mu=X[:,0]*beta, sigma=sigma_i, observed=y)

    # Sampling
    trace = pm.sample(1000, target_accept=0.9, return_inferencedata=True)

# Identify outliers based on the posterior of lambda_i
lambda_posterior = trace.posterior['lambda'].values
lambda_mean = np.mean(lambda_posterior, axis=(0, 1))
outlier_threshold = np.quantile(lambda_mean, 0.05)
outliers = np.where(lambda_mean < outlier_threshold)[0]
print(f"Potential outliers: {outliers}")



Ridge Regression Loss: 215.93613136787422
Lasso Regression Loss: 215.90725352689697
Bayesians do not optimize posterior distributions, they sample from them; but, the posterior distributions are nonetheless 'regularizations' of the likelihood through the prior.


Potential outliers: [ 6 13 25 67 79]
